In [5]:
# jupyter nbconvert "TCXP - MAPI.ipynb" --to slides --post serve

# PENDING: references 

# TCXP - A Scalable Algorithm for Explaining Individual Tree-based Classifier Predictions

[mateo.restrepo@yuxiglobal.com](mailto:mateo.restrepo@yuxiglobal.com) - Head of Data Analytics at Yuxi Global, Medellín

<div style="margin:auto"> 
    <p text-align="center">
       <img style="display:block;margin-left:auto;margin-right:auto;" width="450px" height="350px" src="ml_scratching.png" />
    </p>
</div>

# Outline 

* Context and terminology  ...
* Explanations??? Please explain yourself!
* What is TCXP? 
* TCXP vs. LIME 
* Demo on real data


###  Context and terminology

A   **(hard) binary classifier** is just a (computable) function $C$ that takes a vector of covariates (features) and outputs a a result in $\{0,1\}$

$$  \begin{align} C & : & \mathbb R^f  & \rightarrow & \{ -1 , +1\} \\
& \, & \mathbf x  &\mapsto & y 
\end{align}$$ 


A **(soft) binary classifier** is : 

$$ 
\begin{align} p : & \mathbb  R^f & \rightarrow &\,  [0, 1] \\
    & \mathbf x &  \mapsto &  p^{+}(\mathbf x)   
\end{align}
$$


$p^+(\mathbf{x})$ is interpreted as the (estimated) probability that $\mathbf x$ belongs to the _positive class_.

  
 **(supervised) Machine learning** is the ~~science~~ _art_ of automatically constructing an optimal $C$ (or $p^+$) from many examples $\{(\mathbf x^{(i)}, y^{(i)}) \, :\, i=1,\dots, N \}$.   
  

## Examples from  industry


  *  **Credit risk:** $\mathbf{x}$ = information about a customer and a credit product  $p^+(\mathbf x)$ is the probability that she will default.
  
  
  *  **Customer churn:**  $\mathbf{x}$ = information about a customer's behavior  $p^+(\mathbf x)$ is the probability that he will stop being my client.
  
  
  *  **Online-advertisement:**  $\mathbf{x}$ = information about an online ad and a person  that is looking at it  $p^+(\mathbf 
  x)$  the probability that person will click on it
  
  

## Explanations (or lack thereof) in the context of ML models


* Explanation for prediction: 
    * Answer the question: **Why** did the model predict $\hat{y}^{(i)}$ on input $\mathbf x^{(i)}$?
    

    
* In industries (such as banking, insurance):
    * Sales staff sometimes ask about individual predictions...
    * Predictive analytics promises **actionable insights**:
        * Individual prediction $\rightarrow$ individual action 
       

* **One way to implement**: Quantify each *feature*'s contribution to the prediction?
* Something like this: 
  
<img height="8000" width="4000" src="explanation_bar_plot.png"/>    



## Explaining  advanced ML algorithms to sales staff

<div>
    <p>        
    <img height="400px" width="250" src="confused-girl.jpg"      style="display: inline-block"/>
    <img height="250" width="250" src="confused-boy.jpg"        style="display: inline-block" />
    <img height="250" width="250" src="confused-black-girl.jpg" style="display: inline-block" />
    </p>
</div>

## Besides... You need to produce explanations BY LAW!


* _Algorithmic Fairness Provisions_ of the  **General Data Protection regulation (GDPR)**:


  *  _"The Right to Explanation of Automated Decision mandates that the data subject has a right to get an explanation about decisions made by algorithms and a right to opt-out of some algorithmic decisions altogether if they are not satisfied with it."_ 
  
  
  *  [Article: Deep Learning going illegal in Europe](https://www.analyticsindiamag.com/deep-learning-going-illegal-europe/)
  

## A dichotomy

*  Predictions by *simple algoritms* (e.g  for **Logistic regression**) are "easy" to explain.


    
* Predictions by *advanced algorithms*, e.g. random forests, neural networks, XGBoost are *hard* to explain
    * black-box nature and high internal complexity of these models

<div style="margin:auto">
    <p text-align="center">
    <img style="margin-left:auto;margin-right:auto;display:block" src="truth-and-clarity.png" />
    </p>
</div>

# What is TCXP?


* An algo to generate **interpretable explanations** for *individual* tree-based classifier predictions.

  -  **Simple** and **scalable**
  
  

  
* **Definition:** An **explanation** fo an individual prediction   $p^+( \mathbf{x}(i))$:
    $$( p_0(i), \Delta p_{1}(i), \dots, \Delta p_{f}(i)  )  \quad \text{ such that }\quad  
     p_{0}(i) + \sum_{j=1}^f \Delta p_{j}(i) = p^+  ( \mathbf x ^{(i)} ) $$
$\Delta p_j(i)$ is interpreted as the _contribution_ to the prediction coming from the $j$-th feature, $x_j(i)$.     

  

* **How**?
  * **Basic idea:**  carry out *careful accounting* of probability contributions of each variable.

   

## Binary classification through a tree: leaf counts
<div> 
    <img  height="800px" width="870px" src="tree_00.png" />    
</div>

A classification trees has **internal decision** nodes, each using a single variable, and final (non-decision) **leaves**

For each leaf node $k$ we record count of positive class over total count: $( n^+_k ,\, n^+_k + n^-_k )  $

## Binary classification through a tree: probability estimates


<div> 
    <img  height="800px" width="870px" src="tree_01.png" />    
</div>

For each leaf $k$, $p^+_k = n^+_k / (n^+_k + n^-_k) $

## Explanation generation: all node counts 
    
<div>
    <img  height="800px" width="870px"  src="tree_02.png" />    
</div>


For each internal node compute  $( n^+_k, \, (n^+_k + n^-_k)) $

## Explanation generation: all node probs
    
<div>
    <img  height="800px" width="870px" src="tree_03.png" />    
</div>


For each internal node node $k$, $p^+_k = n^+_k / (n^+_k + n^-_k) $

## Explanation generation: deltas on all edges
    
<div>
    <img    height="800px" width="870px" src="tree_04.png" />    
</div>
If $k$ is the parent of $l$,  compute $\Delta p^+ _{(k,l)} := p^+_l - p^+_k$

## Explanation generation: assigning deltas to variables -  Case 1
    
<div>
    <img  height="800px" width="870px" src="tree_c1.png" />    
</div>

First delta is attributable to $Y$, second to $Z$, third to $X$

## Explanation generation: assigning deltas to variables -  Case 2
    
<div>
    <img  height="800px" width="870px" src="tree_c2.png" />    
</div>

First delta is attributable to $Y$, second to $X$, third to $Y$ again.

## Explanation generation: assigning deltas to variables -  Case 3
    
<div>
<img  height="800px" width="870px" src="tree_c3.png" />    
</div>


First delta is attributable to $Y$, second to $X$, none to $Z$

## Extension to forest / GBT and complexity

Consider a clasifier $\mathcal{T}$, consisting of $T$ trees, with weights $w_1, w_2, \dots, w_T$ <br />
with ($\sum^T_{i=1} w_i = 1$)  and let 
$$\mathbf{e}^{(t)}(i) := ( p_0^{(t)}(i), \Delta p^{(t)}_{1}(i), \dots, \Delta p^{(t)}_{f}(i)  )$$ denote the explanation computed on tree $t$, $t=1, \dots, T$. 

Let the explanation for the whole classifier be the weighted sum:
$$\mathbf{e}^{(\mathcal T)}(i) := \sum_{i=1}^T w_i \mathbf{e}^{(t)}(i) $$

### Complexity

**Theorem**: If the trees in $\mathcal T$ consist of a total of $V$ nodes and the maximum depth of any tree is $d$, then:

  * Precomputing cond. probabilitities at all nodes takes $O(V)$ time and $O(V)$ space.
  * Computing an individual explanation takes $O(d)$ time and $O(f)$ space
  
**Proof:** Trivial.

**Corollary**: both bounds are dominated by the corresponding bounds of training the classifier and generating a prediction

## Another approach for explanation generation: LIME

**LIME**: **L**ocal **I**nterpreatable **M**odel-agnostic **E**xplanations

**Basic Idea**: For each $\mathbf x (i)$:
  * Generate (100s of) random samples of a neighborhood around $\mathbf x(i)$.
  * Compute prediction using model $M$ for each sample.
  * Fit linear ML model to predictions.
  * Cast coefficients of linear model as variable importances.
  

**Main drawback**:
   * It is **very slow** to compute an explanation for a single datapoint! 
     * $\rightarrow$ This doesn't scale!
 

## Comparison between LIME and TCXP

<div>
    <img  width="600px" height="650px" src="lime_vs_tcxp.png" />    
</div>

## Extras  

  * Visit our company's website: [www.yuxiglobal.com](www.yuxiglobal.com)

  * Add me on Linked-In: https://www.linkedin.com/in/mateorestrepo/

  * Get the codez! https://github.com/YuxiGlobal/data-analytics

  *  LIME: 
    * https://arxiv.org/abs/1602.04938
    * https://github.com/marcotcr/lime

